### Программа для сравнения отчета Splunk для инвойсов с данными BSS

### Запрос для выгрузки данных из BSS для сверки

SET @start_date = 20190201000000;
SET @end_date = 20190301000000;

SELECT
REP_DATE,Customer,USAGE_SOURCE
,SUM(Cost) AS Cost
,SUM(Revenue) AS Revenue
FROM
(
SELECT DATE_FORMAT(ev.TreatedTimestampUtc, "%Y-%m") AS REP_DATE
,s.Customer
,SUM(Cost) AS Cost
,SUM(Revenue) AS Revenue
,'Mobile' as USAGE_SOURCE
FROM TAP.SUBSCRIBER s, TAP.MOC ev FORCE INDEX (ind_treated_moc)
WHERE ev.Subscriber = s.Id
AND ev.TreatedTimestampUtc >= @start_date AND ev.TreatedTimestampUtc < @end_date
GROUP BY REP_DATE,s.Customer,USAGE_SOURCE

UNION ALL

SELECT DATE_FORMAT(ev.TreatedTimestampUtc, "%Y-%m") AS REP_DATE
,s.Customer
,SUM(Cost) AS Cost
,SUM(Revenue) AS Revenue
,'Mobile' as USAGE_SOURCE
FROM TAP.SUBSCRIBER s, TAP.MTC ev FORCE INDEX (ind_treated_mtc)
WHERE ev.Subscriber = s.Id
AND ev.TreatedTimestampUtc >= @start_date AND ev.TreatedTimestampUtc < @end_date
GROUP BY REP_DATE,s.Customer,USAGE_SOURCE

UNION ALL

SELECT DATE_FORMAT(ev.TreatedTimestampUtc, "%Y-%m") AS REP_DATE
,s.Customer
,SUM(Cost) AS Cost
,SUM(Revenue) AS Revenue
,'Mobile' as USAGE_SOURCE
FROM TAP.SUBSCRIBER s, TAP.GPRS_CALL ev FORCE INDEX (ind_treated_gprs)
WHERE ev.Subscriber = s.Id
AND ev.TreatedTimestampUtc >= @start_date AND ev.TreatedTimestampUtc < @end_date
GROUP BY REP_DATE,s.Customer,USAGE_SOURCE
) t GROUP BY REP_DATE,Customer,USAGE_SOURCE

In [73]:
import pandas as pd
from os.path import join
import roamability as rb

downloads = r'c:\Users\balob\Downloads\DATA'
file_bss_mobile = 'BSS_MOBILE_REPORT_1910.csv'
file_splunk_invoice = 'Monthly_auto_invoice_report-2019-11-01.csv'
# file_splunk_invoice = '9-2019 Invoice Summary 01.11.2019, 12-50-04.csv' # BSS report from UI

In [74]:
# Загружаем данные BSS по Mobile

df_bss_mobile = pd.read_csv(join(downloads, file_bss_mobile), sep=';')
df_bss_mobile.drop('USAGE_SOURCE', axis=1, inplace=True)
df_bss_mobile.drop('REP_DATE', axis=1, inplace=True)
df_bss_mobile.Revenue = df_bss_mobile.Revenue.round().astype('int')
df_bss_mobile.head(3)

,Customer,Cost,Revenue
0,Cellact,34.153517,71
1,Clay RT,103.029301,420
2,Cloud 9 TAP,21376.121043,42747


In [3]:
# Вызгрузить данныхе по Voip из BSS

sql_srt='''SELECT
DATE_FORMAT(ev.CallTimeUtc, "%Y-%m") AS REP_DATE
,s.Customer
,SUM(Cost) AS Cost
,SUM(Revenue) AS Revenue
,'VOIP' as USAGE_SOURCE
FROM TAP.SUBSCRIBER s, voip.CALLS ev
WHERE ev.SponsImsi = s.SponsIMSI
AND ev.CallTimeUtc >= 20191001000000 AND ev.CallTimeUtc < 20191101000000
GROUP BY REP_DATE,s.Customer,USAGE_SOURCE'''

# AND ev.CallTimeUtc >= 20190331220000 AND ev.CallTimeUtc < 20190430220000

with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df_bss_voip = pd.read_sql_query(sql_srt, cnxn)

df_bss_voip.Revenue = df_bss_voip.Revenue.round() #.astype('int')
df_bss_voip.drop('USAGE_SOURCE', axis=1, inplace=True)
df_bss_voip.drop('REP_DATE', axis=1, inplace=True)
df_bss_voip.head(3)

,Customer,Cost,Revenue
0,Cellact,0.49092,2.0
1,Clay RT,20.97349,33.0
2,Dialoq - Genesisclub RT,0.00000,0.0


In [75]:
# Объединяем Mobile и Voip

df_bss = pd.merge(df_bss_mobile, df_bss_voip, how='outer', on='Customer', suffixes=['_bss_mobile', '_bss_voip'])
df_bss.head(3)

,Customer,Cost_bss_mobile,Revenue_bss_mobile,Cost_bss_voip,Revenue_bss_voip
0,Cellact,34.153517,71.0,0.49092,2.0
1,Clay RT,103.029301,420.0,20.97349,33.0
2,Cloud 9 TAP,21376.121043,42747.0,NaN,NaN


In [76]:
# Загружаем данные Splunk

df_splunk_invoice = pd.read_csv(join(downloads, file_splunk_invoice))
df_splunk_invoice.head(3)

,Customer,Mobile Revenue,VoIP Revenue,Discount,Discount Amount,MonthlyFee,Total Invoice charge
0,Cellact,71,2,0.00,-0.0,0,73.0
1,Clay RT,420,33,0.00,-0.0,0,453.0
2,Cloud 9 TAP,42747,0,0.15,-6412.0,3700,40035.0


In [69]:
# Коррекция файла из BSS. Не использовать при отчете из Splunk

df_splunk_invoice['Mobile Revenue'] = df_splunk_invoice['Mobile Revenue'].round()
df_splunk_invoice['VoIP Revenue'] = df_splunk_invoice['VoIP Revenue'].round()
df_splunk_invoice.replace(to_replace={r'TAP':'', 'RT':'', 'DMI':'', ' ':''}, regex=True, inplace=True)
df_splunk_invoice.Customer = df_splunk_invoice.Customer.str.strip(' ')
df_splunk_invoice.Customer = df_splunk_invoice.Customer.str.lower()

df_bss.Customer.replace(to_replace={r'TAP':'', 'RT':'', 'DMI':'', ' ':''}, regex=True, inplace=True)
df_bss.Customer = df_bss.Customer.str.strip(' ')
df_bss.Customer = df_bss.Customer.str.lower()
df_bss

,Customer,Cost_bss_mobile,Revenue_bss_mobile,Cost_bss_voip,Revenue_bss_voip
0,cellact,34.153517,71.0,0.49092,2.0
1,clay,103.029301,420.0,20.97349,33.0
2,cloud9,21376.121043,42747.0,NaN,NaN
3,dent,5.369143,11.0,NaN,NaN
4,dialoq-genesisclub,1327.895539,2881.0,0.00000,0.0
5,drimsim,4.306812,9.0,1.42828,2.0
6,freecell,638.967676,1292.0,123.06251,195.0
7,joytelecom,1115.293542,2233.0,NaN,NaN
8,joy_telecom,951.009117,1904.0,NaN,NaN
9,justworksmobile,159.388095,319.0,0.37862,0.0


In [77]:
df_bss.Revenue_bss_voip.sum()

358.0

In [78]:
df_splunk_invoice['VoIP Revenue'].sum()

324

In [79]:
# Делаем сравнительный отчет

df_compare = pd.merge(df_splunk_invoice, df_bss, how='outer', on='Customer')
df_compare.drop(['Discount', 'Discount Amount', 'MonthlyFee', 'Total Invoice charge', 'Cost_bss_mobile', 'Cost_bss_voip'],
                axis=1, inplace=True)

df_compare.fillna(0, inplace=True)
df_compare.head(3)

,Customer,Mobile Revenue,VoIP Revenue,Revenue_bss_mobile,Revenue_bss_voip
0,Cellact,71.0,2.0,71.0,2.0
1,Clay RT,420.0,33.0,420.0,33.0
2,Cloud 9 TAP,42747.0,0.0,42747.0,0.0


In [80]:
# Несоответствия в Mobile

df_compare.loc[df_compare['Mobile Revenue'] != df_compare.Revenue_bss_mobile,
               ['Customer', 'Mobile Revenue', 'Revenue_bss_mobile']]

,Customer,Mobile Revenue,Revenue_bss_mobile
7,Joy telecom,7.0,2233.0
8,Joy_telecom,4130.0,1904.0


In [11]:
4130.0 + 7.0 # 4137.0
2233.0 + 1904.0

4137.0

In [12]:
# Несоответствия в Voip

df_compare.loc[df_compare['VoIP Revenue'] != df_compare.Revenue_bss_voip,
              ['Customer', 'VoIP Revenue', 'Revenue_bss_voip']]

,Customer,VoIP Revenue,Revenue_bss_voip
6,Freecell,193.0,195.0
22,Roamability Resellers Test,0.0,12.0
23,Roamability Test Orange Sponsor RT,1.0,0.0
24,Roamability Test RT,18.0,31.0
39,Roamability Test MB Sponsor RT,0.0,7.0
40,Roamability Test SMART Sponsor RT,0.0,1.0
